In [114]:
import sys
import os

if not [ k for k in sys.path if 'nvdbapi' in k]:
    print( "Legger NVDB api til søkestien")
    sys.path.append('C:\\Users\\Kristin\\OneDrive\\nvdbapi-V3-master')

In [115]:
import pandas as pd
import geopandas as gpd
from shapely import wkt 

#import STARTHER
import nvdbapiv3

import hvplot.pandas
from holoviews import opts
import cartopy.crs as ccrs


#Henter kurvatur, horisontalelement
#Henter objekttype 639 ver å bruke nvdbFagdata
sokeobjekt_sving = nvdbapiv3.nvdbFagdata( 639 )

kommunenummer = 5001

#Filtrerer søket på bestemt kommune
#sokeobjekt_sving.filter( { 'vegsystemreferanse' : 'Ev,Rv'})
sokeobjekt_sving.filter( {'kommune' : kommunenummer })

#Gjør om til geodataframe
#"Sving" er nå en tabell, hvor man kan hente ut ulik infrmasjon
vegstrekning = pd.DataFrame( sokeobjekt_sving.to_records() )
vegstrekning['geometry'] = vegstrekning['geometri'].apply( lambda x : wkt.loads( x ))
vegstrekning = gpd.GeoDataFrame( vegstrekning, geometry='geometry', crs=5973 )

Eksport av 13915 objekter kommer til å ta tid...
Objekt 1000 av 13915
Objekt 5000 av 13915
Objekt 10000 av 13915


In [116]:
#vegstrekning.style.set_table_attributes("style='display:inline'").set_caption('Overskrift tabell ')
vegstrekning

,objekttype,nvdbId,versjon,startdato,Type,Radius,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,...,vegnummer,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Beregningsinfo,Klotoideparameter,geometry
0,639,323142728,5,2021-06-25,Rettlinje,0,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72299,Vegtrase,Kanalisert veg,...,704,0.483813,0.495062,4.869,Nei,K,"LINESTRING Z(267745.638 7031137.249 151.733, 2...",NaN,NaN,"LINESTRING Z (267745.638 7031137.249 151.733, ..."
1,639,323142763,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,704,0.000000,0.449511,31.680,Nei,K,"LINESTRING Z(267965.2 7031090.3 157.233, 26796...",NaN,NaN,"LINESTRING Z (267965.200 7031090.300 157.233, ..."
2,639,323142764,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,704,0.449511,0.849923,28.151,Nei,K,"LINESTRING Z(267984.804 7031100.819 157.033, 2...",NaN,NaN,"LINESTRING Z (267984.804 7031100.819 157.033, ..."
3,639,323142765,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,704,0.849923,1.000000,10.536,Nei,K,"LINESTRING Z(267963.394 7031100.277 157.233, 2...",NaN,NaN,"LINESTRING Z (267963.394 7031100.277 157.233, ..."
4,639,323143476,1,2012-02-13,Sirkelbue,-59,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72313,Vegtrase,Kanalisert veg,...,707,0.000000,0.000747,7.430,Nei,K,"LINESTRING Z(260404.6 7044175.9 30.331, 260400...",NaN,NaN,"LINESTRING Z (260404.600 7044175.900 30.331, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15094,639,1013893577,1,2021-08-04,Sirkelbue,231,NaN,42681,Vegtrase og kjørebane,Enkel bilveg,...,7830,0.665338,0.697986,50.132,Nei,K,"LINESTRING Z(266705.628 7038910.183 218.871, 2...","bd=20210804, kv=3.0.0.20, MKM2, ah=0,5",0.0,"LINESTRING Z (266705.628 7038910.183 218.871, ..."
15095,639,1013902269,1,2021-08-06,Sirkelbue,-29,NaN,1842943,Vegtrase og kjørebane,Enkel bilveg,...,706,0.979565,0.996520,14.616,Nei,K,"LINESTRING Z(268869.419 7041498.456 13.627, 26...","bd=20210806, kv=3.0.0.20, MKM2, ah=0,5",0.0,"LINESTRING Z (268869.419 7041498.456 13.627, 2..."
15096,639,1013902270,1,2021-08-06,Sirkelbue,-408,NaN,1842943,Vegtrase og kjørebane,Enkel bilveg,...,706,0.892423,0.979565,75.120,Nei,K,"LINESTRING Z(268925.972 7041449.298 10.683, 26...","bd=20210806, kv=3.0.0.20, MKM2, ah=0,5",0.0,"LINESTRING Z (268925.972 7041449.298 10.683, 2..."
15097,639,1013902271,1,2021-08-06,Sirkelbue,-473,NaN,1842943,Vegtrase og kjørebane,Enkel bilveg,...,706,0.619247,0.892423,235.489,Nei,K,"LINESTRING Z(269037.595 7041244.95 18.595, 269...","bd=20210806, kv=3.0.0.20, MKM2, ah=0,5",0.0,"LINESTRING Z (269037.595 7041244.950 18.595, 2..."


In [117]:
#Henter ut verdier fra "Radius" kolonnen. describe()-metoden skriver ut ulik statistikk for den gitte kolonnen.
vegstrekning['Radius'].describe()

count    15099.000000
mean        25.123386
std       1219.293288
min      -9999.000000
25%        -81.000000
50%          0.000000
75%         93.000000
max       9999.000000
Name: Radius, dtype: float64

In [118]:
#Finner antall av hvert element i kolonnen "Type".
vegstrekning['Type'].value_counts()

Sirkelbue    12388
Rettlinje     2711
Name: Type, dtype: int64

In [119]:
#Ønsker bare å se på vegstrekninger som er sirkelbuer, så setter Type-kolonnen lik "sirkelbue". Endrer så navn fra "vegstrekning"
#til "sving"
sving = vegstrekning[ vegstrekning['Type'] == 'Sirkelbue' ].copy()

#Henter så ut informasjon om radius for de veistrekningene som er sirkelbuer
sving['Radius'].describe()

count    12388.000000
mean        30.621408
std       1346.060708
min      -9999.000000
25%       -124.000000
50%         -4.000000
75%        143.000000
max       9999.000000
Name: Radius, dtype: float64

In [120]:
sving['Radius_abs'] = sving['Radius'].abs()
sving['Radius_abs'].describe()

count    12388.000000
mean       493.091863
std       1252.859913
min          1.000000
25%         47.000000
50%        134.000000
75%        395.000000
max       9999.000000
Name: Radius_abs, dtype: float64

In [121]:
#Ønsker bare å se på de krappe svingene, så finner alle svinger som har radius mindre enn 180. Henter så ut statistikk for disse.
krappsving = sving[ sving['Radius_abs'] < 180]
krappsving['Radius_abs'].describe()

count    7154.000000
mean       67.489796
std        48.796134
min         1.000000
25%        25.000000
50%        57.000000
75%       103.000000
max       179.000000
Name: Radius_abs, dtype: float64

In [122]:
krappsving[ krappsving['Radius_abs'] == 0]

,objekttype,nvdbId,versjon,startdato,Type,Radius,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,...,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Beregningsinfo,Klotoideparameter,geometry,Radius_abs


In [123]:
#Fjerner svinger som har radius lik 0
tmp = krappsving[ krappsving['Radius_abs'] == 0]

#Henter bare svingene som har radius større enn 0
if len( tmp) > 0: 
    tmp.hvplot( crs=ccrs.UTM(zone=33), hover_cols=['trafikantgruppe', 'vref', 'segmentlengde'], colorbar=False, tiles='OSM',
          line_width=14)
    
krappsving = krappsving[ krappsving['Radius_abs'] != 0]
krappsving.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='krappesvinger', driver='GPKG')

# Steg 2: Lager buffer rundt krappe svinger

In [124]:
#Definerer ny variabel "svingebuffer" som er en kopi av "krappsving"
svingebuffer = krappsving.copy()
svingebuffer['geometry'] = svingebuffer['geometry'].buffer( 200 ) # 200 meters radius


In [125]:
tmp3 = svingebuffer.iloc[0:5]
tmp3.hvplot( crs=ccrs.UTM(zone=33), colorbar=False, tiles='OSM')

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [126]:
krappsving.iloc[0:5].hvplot(crs=ccrs.UTM(zone=33), colorbar=False, tiles='OSM', line_width=5 )

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [127]:
svingebuffer['vegnr'] = svingebuffer['vref'].apply( lambda x: x.split()[0] )
svingebuffer['vegnr']

1         FV704
2         FV704
3         FV704
4         FV707
5        KV9432
          ...  
15080    KV7830
15081    KV7830
15084    KV7830
15092    KV7830
15095     RV706
Name: vegnr, Length: 7154, dtype: object

In [128]:
svingebuffer_dissolve = svingebuffer.dissolve(by='vegnr')
svingebuffer.head(3)

,objekttype,nvdbId,versjon,startdato,Type,Radius,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,...,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Beregningsinfo,Klotoideparameter,geometry,Radius_abs,vegnr
1,639,323142763,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,0.449511,31.680,Nei,K,"LINESTRING Z(267965.2 7031090.3 157.233, 26796...",NaN,NaN,"POLYGON ((267880.635 7031271.448, 267882.254 7...",11,FV704
2,639,323142764,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,0.849923,28.151,Nei,K,"LINESTRING Z(267984.804 7031100.819 157.033, 2...",NaN,NaN,"POLYGON ((267770.595 7031152.912, 267769.976 7...",11,FV704
3,639,323142765,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,1.000000,10.536,Nei,K,"LINESTRING Z(267963.394 7031100.277 157.233, 2...",NaN,NaN,"POLYGON ((268161.382 7031128.006, 268162.875 7...",11,FV704


In [129]:
svingebuffer_dissolve.reset_index( inplace=True)
# Vi liker bedre at vegnr er en vanlig kolonne fremfor at det blir brukt som indeks. 
# Merk at her bruker vi IKKE nøkkelord `drop=True`, for her vil vi at indeksen settes inn blant kolonnene
svingebuffer.head(3)

,objekttype,nvdbId,versjon,startdato,Type,Radius,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,...,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Beregningsinfo,Klotoideparameter,geometry,Radius_abs,vegnr
1,639,323142763,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,0.449511,31.680,Nei,K,"LINESTRING Z(267965.2 7031090.3 157.233, 26796...",NaN,NaN,"POLYGON ((267880.635 7031271.448, 267882.254 7...",11,FV704
2,639,323142764,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,0.849923,28.151,Nei,K,"LINESTRING Z(267984.804 7031100.819 157.033, 2...",NaN,NaN,"POLYGON ((267770.595 7031152.912, 267769.976 7...",11,FV704
3,639,323142765,1,2012-02-13,Sirkelbue,-11,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72301,Vegtrase og kjørebane,Rundkjøring,...,1.000000,10.536,Nei,K,"LINESTRING Z(267963.394 7031100.277 157.233, 2...",NaN,NaN,"POLYGON ((268161.382 7031128.006, 268162.875 7...",11,FV704


# Steg 3 - henter stigningsdata

In [130]:
#Gjør videre det samme som over, men for stigningsdata. Dvs. henter objekttype for stigning, filtrerer på kommune, oppretter en 
#tabell.

sokeobjekt_stigning = nvdbapiv3.nvdbFagdata( 825 )
sokeobjekt_stigning.filter(  { 'egenskap' : '9396 > 6 OR 9396 < -6'  }  )
sokeobjekt_stigning.filter( {'kommune' : kommunenummer })
stig = pd.DataFrame( sokeobjekt_stigning.to_records( ) )

#Oppretter geodataframe og lagrer til fil
stig['geometry'] = stig['geometri'].apply( lambda x : wkt.loads( x ))
stig = gpd.GeoDataFrame( stig, geometry='geometry', crs=5973 )  
stig.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='brattebakker', driver='GPKG' )

#Lager buffer rundt områder med stor stigning (+- 6 %) (Buffer har radius 500 - samme som buffer rundt krappe svinger (steg 2))
stigebuffer = stig.copy()
stigebuffer['geometry'] = stigebuffer['geometry'].buffer( 200 ) # 200 meters radius

#Splitter opp vegnummer på mellomrom
stigebuffer['vegnr'] = stigebuffer['vref'].apply( lambda x: x.split()[0] )

stigebuffer_dissolve = stigebuffer.dissolve( by='vegnr')
stigebuffer_dissolve.reset_index( inplace=True )

stigebuffer_dissolve2 = stigebuffer_dissolve.explode( )

stigebuffer_dissolve2.reset_index( inplace=True, drop=True )

stigebuffer_dissolve2.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='bakkebuffer', driver='GPKG' )

C:\Users\Kristin\OneDrive\nvdbapi-V3-master\nvdbapiv3\nvdbapiv3.py:438: UserWarning: 
You should provide the file nvdbapi-clientinfo.json 
 
{ "X-Client" : "YOUR SYSTEM",
 "X-Kontaktperson" : "ola.nordmann@eposten.din" }

  warn( mytext )


# Steg 5

In [131]:
#Finner områdene som er svingete og bratt, det vil si hvor disse overlapper (intersection)
brattsving = gpd.overlay( stigebuffer_dissolve, svingebuffer_dissolve, how='intersection')
brattsving.dtypes

vegnr_1                  object
objekttype_1              int64
nvdbId_1                  int64
versjon_1                 int64
startdato_1              object
Stigning                float64
relasjoner_1             object
veglenkesekvensid_1       int64
detaljnivå_1             object
typeVeg_1                object
kommune_1                 int64
fylke_1                   int64
vref_1                   object
veglenkeType_1           object
vegkategori_1            object
fase_1                   object
vegnummer_1               int64
startposisjon_1         float64
sluttposisjon_1         float64
segmentlengde_1         float64
adskilte_lop_1           object
trafikantgruppe_1        object
geometri_1               object
Beregningsinfo_1         object
vegnr_2                  object
objekttype_2              int64
nvdbId_2                  int64
versjon_2                 int64
startdato_2              object
Type                     object
Radius                    int64
relasjon

In [132]:
#Sjekker om man har fått overlapp på ulike vegnummer. Disse er datafeil, så disse må kastes.
brattsving[ brattsving['vegnr_1'] != brattsving['vegnr_2']][['vegnr_1', 'vegnr_2']]

,vegnr_1,vegnr_2
1,FV6658,EV6
2,FV6686,EV6
3,FV950,EV6
4,KV1006,EV6
5,KV1034,EV6
...,...,...
8817,PV4460,KV7100
8818,PV5195,KV4295
8819,PV5886,KV4295
8820,PV7705,KV6950


In [133]:
#"Brattsving"-variabelen tar bare med tilfeller hvor vegnummer er det samme (reduserer tabellen)
brattsving = brattsving[ brattsving['vegnr_1'] == brattsving['vegnr_2']]
brattsving

,vegnr_1,objekttype_1,nvdbId_1,versjon_1,startdato_1,Stigning,relasjoner_1,veglenkesekvensid_1,detaljnivå_1,typeVeg_1,...,startposisjon_2,sluttposisjon_2,segmentlengde_2,adskilte_lop_2,trafikantgruppe_2,geometri_2,Beregningsinfo_2,Klotoideparameter,Radius_abs,geometry
0,EV6,825,323180580,1,2012-01-31,6.8,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",72197,Vegtrase og kjørebane,Rampe,...,0.000000,0.009455,6.000,Mot,K,"LINESTRING Z(270261.4 7038513.8 37.339, 270259...","bd=20190704, kv=1.0.0.49, MKM2, ah=0,7",0.0,6,"MULTIPOLYGON (((269127.717 7036516.223, 269116..."
64,FV6658,825,1013693922,1,2021-06-25,7.1,None,42694,Vegtrase,Kanalisert veg,...,0.000000,0.008724,7.708,Nei,K,"LINESTRING Z(271054.1 7039854.8 46.137, 271056...","bd=20210625, kv=3.0.0.18, MKM2, ah=0,5",0.0,74,"MULTIPOLYGON (((271608.197 7038759.909, 271627..."
94,FV6660,825,323164448,1,2012-02-13,8.6,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",42837,Vegtrase og kjørebane,Enkel bilveg,...,0.045931,0.055056,29.897,Nei,K,"LINESTRING Z(282000.254 7032196.337 176.838, 2...","bd=20210625, kv=3.0.0.18, MKM2, ah=0,5",0.0,93,"MULTIPOLYGON (((283551.175 7031816.866, 283568..."
109,FV6682,825,323164937,1,2012-02-13,14.9,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1718301,Vegtrase og kjørebane,Rundkjøring,...,0.044576,0.050066,25.065,Nei,K,"LINESTRING Z(267850.746 7033328.554 138.145, 2...","bd=20210628, kv=3.0.0.18, MKM2, ah=0,5",0.0,126,"MULTIPOLYGON (((265979.932 7030897.354, 265999..."
139,FV950,825,1013692903,1,2021-06-25,6.1,None,41619,Vegtrase og kjørebane,Enkel bilveg,...,0.995567,0.996834,2.038,Nei,K,"LINESTRING Z(276368.7 7040694.7 30.841, 276370...","bd=20210625, kv=3.0.0.18, MKM2, ah=0,5",0.0,121,"POLYGON ((277379.727 7040562.550, 277385.806 7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8813,PV1405,825,323364443,1,2012-02-10,-7.6,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",72681,Vegtrase,Kanalisert veg,...,0.000000,0.000964,3.726,Nei,K,"LINESTRING Z(270951.5 7028647.5 173.431, 27095...",None,NaN,15,"POLYGON ((270804.238 7028783.262, 270818.042 7..."
8814,PV3132,825,323210505,1,2012-01-25,-7.1,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1945404,Vegtrase og kjørebane,Enkel bilveg,...,0.000000,0.214286,24.008,Nei,K,"LINESTRING Z(269318.6 7032607.2 136.727, 26932...",None,NaN,55,"POLYGON ((269175.944 7032467.305, 269169.226 7..."
8821,PV7705,825,323213780,1,2012-01-25,6.8,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",42663,Vegtrase og kjørebane,Enkel bilveg,...,0.953259,0.969813,16.872,Nei,K,"LINESTRING Z(273727.253 7041013.131 67.837, 27...",None,NaN,136,"POLYGON ((273590.803 7041158.359, 273606.362 7..."
8822,PV7872,825,323213883,1,2012-01-25,10.9,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",42689,Vegtrase og kjørebane,Enkel bilveg,...,0.436782,0.793103,31.391,Nei,K,"LINESTRING Z(266522.451 7032580.352 145.006, 2...",None,NaN,109,"POLYGON ((266385.795 7032420.268, 266369.019 7..."


In [134]:
#Fjerner kolonner som en ikke trenger

brattsving = brattsving[[ 'vegnr_1', 'geometry' ] ].copy()
brattsving.rename( columns={ 'vegnr_1' : 'vegnr' }, inplace=True)

brattsving2 = brattsving.explode()

brattsving2.reset_index( inplace=True, drop=True )

brattsving.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='overlay_brattogkrapt', driver='GPKG' )

brattsving2

,vegnr,geometry
0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7..."
1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7..."
2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7..."
3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7..."
4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7..."
...,...,...
555,PV1405,"POLYGON ((270804.238 7028783.262, 270818.042 7..."
556,PV3132,"POLYGON ((269175.944 7032467.305, 269169.226 7..."
557,PV7705,"POLYGON ((273590.803 7041158.359, 273606.362 7..."
558,PV7872,"POLYGON ((266385.795 7032420.268, 266369.019 7..."


In [135]:
# Plotter inntil de 15 første
brattsving2.iloc[0:min( 15, len( brattsving2)-1 )].hvplot(  crs=ccrs.UTM(zone=33), 
                     tiles='OSM', hover_cols=['vegnr'])

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]   (vegnr)

In [136]:
brattsving2.reset_index( inplace=True )
brattsving2.rename( columns={'index' : 'minFineId'}, inplace=True)
brattsving2
#MinFineID er en unik indeks som gjør det lettere å hente ut data senere

,minFineId,vegnr,geometry
0,0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7..."
1,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7..."
2,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7..."
3,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7..."
4,4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7..."
...,...,...,...
555,555,PV1405,"POLYGON ((270804.238 7028783.262, 270818.042 7..."
556,556,PV3132,"POLYGON ((269175.944 7032467.305, 269169.226 7..."
557,557,PV7705,"POLYGON ((273590.803 7041158.359, 273606.362 7..."
558,558,PV7872,"POLYGON ((266385.795 7032420.268, 266369.019 7..."


In [137]:
#Lager en tabell med soner på hver vei der hvor det er både bratt og krapt.

skitkolonner_framultiIndeks = ['index', 'level_0', 'level_1']

for junk in skitkolonner_framultiIndeks: 
    if junk in brattsving2.columns: 
        brattsving2.drop( columns=[ junk ], inplace=True )
brattsving2

,minFineId,vegnr,geometry
0,0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7..."
1,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7..."
2,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7..."
3,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7..."
4,4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7..."
...,...,...,...
555,555,PV1405,"POLYGON ((270804.238 7028783.262, 270818.042 7..."
556,556,PV3132,"POLYGON ((269175.944 7032467.305, 269169.226 7..."
557,557,PV7705,"POLYGON ((273590.803 7041158.359, 273606.362 7..."
558,558,PV7872,"POLYGON ((266385.795 7032420.268, 266369.019 7..."


# Steg 6 - lager statistikk for krappe svinger innenfor hvert område

In [138]:
#Legger til disse kolonnene i tabellen
sving['Radius_max']    = sving['Radius_abs']
sving['Radius_min']    = sving['Radius_abs']
sving['Radius_mean']   = sving['Radius_abs']
sving['Radius_median'] = sving['Radius_abs']
sving['Radius_antall'] = sving['Radius_abs']

#Joiner to tabeller med spatial join (sjoin()) -- inner join
sving_m_flatedata = gpd.sjoin( sving, brattsving2, how='inner', op='within'  )

sving_m_flatedata

,objekttype,nvdbId,versjon,startdato,Type,Radius,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,...,geometry,Radius_abs,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall,index_right,minFineId,vegnr
4,639,323143476,1,2012-02-13,Sirkelbue,-59,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72313,Vegtrase,Kanalisert veg,...,"LINESTRING Z (260404.600 7044175.900 30.331, 2...",59,59,59,59,59,59,480,480,KV2423
51,639,323144598,2,2021-06-25,Sirkelbue,-902,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72853,Vegtrase,Kanalisert veg,...,"LINESTRING Z (260408.407 7044176.878 30.408, 2...",902,902,902,902,902,902,480,480,KV2423
52,639,323144598,2,2021-06-25,Sirkelbue,-902,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",72853,Vegtrase,Kanalisert veg,...,"LINESTRING Z (260404.600 7044175.900 30.331, 2...",902,902,902,902,902,902,480,480,KV2423
3395,639,323184893,1,2012-01-25,Sirkelbue,125,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",41671,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (260686.382 7044239.302 35.232, 2...",125,125,125,125,125,125,480,480,KV2423
3396,639,323184894,1,2012-01-25,Sirkelbue,86,"{'barn': [{'id': 200844, 'type': {'id': 641, '...",41671,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (260686.369 7044171.727 38.667, 2...",86,86,86,86,86,86,480,480,KV2423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14717,639,1013840471,1,2021-07-22,Sirkelbue,1543,NaN,72684,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (275057.288 7029715.537 151.936, ...",1543,1543,1543,1543,1543,1543,214,214,FV6700
14718,639,1013840474,1,2021-07-22,Sirkelbue,-473,NaN,72684,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (274914.397 7029774.602 151.666, ...",473,473,473,473,473,473,214,214,FV6700
14719,639,1013840475,1,2021-07-22,Sirkelbue,-109,NaN,72684,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (274867.337 7029817.825 147.680, ...",109,109,109,109,109,109,214,214,FV6700
14721,639,1013840481,1,2021-07-22,Sirkelbue,1043,NaN,72684,Vegtrase og kjørebane,Enkel bilveg,...,"LINESTRING Z (274833.894 7029867.697 146.698, ...",1043,1043,1043,1043,1043,1043,214,214,FV6700


In [139]:
#Går gjennom alle veriablene og sier hvilke aggregeringsfunksjon den skal ha.
mineAggregeringsFunksjoner = {  'Radius_max' : 'max', 
                                'Radius_min' : 'min', 
                                'Radius_mean' : 'mean', 
                                'Radius_median' : 'median', 
                                'Radius_antall' : 'count'
}

#Grupperer på minFineID
gruppert_per_flate = sving_m_flatedata.groupby( ['minFineId'] ).agg( mineAggregeringsFunksjoner )

#Grupperer på vegnr og minFineID (tabellen som vises under)
sving_m_flatedata.groupby( ['vegnr', 'minFineId']).agg( mineAggregeringsFunksjoner )

Radius_max  Radius_min  Radius_mean  Radius_median  \
vegnr   minFineId                                                       
EV6     0                3704          39   395.714286          197.0   
        1                1173          25   202.166667           87.0   
        2                1115          10   227.518519           95.0   
        3                1624          11   382.000000          198.0   
        4                1545          11   434.100000          122.5   
...                       ...         ...          ...            ...   
PV8117  204              2069           9   247.578947           30.0   
PV8120  337              9999          17   560.074074           69.0   
PV8707  477              1144           8   232.064516          130.0   
PV8721  338              1151          15   222.629630           97.0   
PV95983 559              1795          30   301.846154          136.5   

                   Radius_antall  
vegnr   minFineId                 
EV6     0                     21  
        1                     36  
        2                     27  
        3                     20  
        4                     10  
...                          ...  
PV8117  204                   19  
PV8120  337                   27  
PV8707  477                   31  
PV8721  338                   27  
PV95983 559                   26  

[549 rows x 5 columns]

In [140]:
#Bruker minFineID som koblingsnøkler når man merger tabellene
brattsving3 = brattsving2.merge( gruppert_per_flate, on='minFineId' )
brattsving3

,minFineId,vegnr,geometry,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall
0,0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7...",3704,39,395.714286,197.0,21
1,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7...",1173,25,202.166667,87.0,36
2,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7...",1115,10,227.518519,95.0,27
3,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7...",1624,11,382.000000,198.0,20
4,4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7...",1545,11,434.100000,122.5,10
...,...,...,...,...,...,...,...,...
544,555,PV1405,"POLYGON ((270804.238 7028783.262, 270818.042 7...",2234,15,428.000000,239.0,10
545,556,PV3132,"POLYGON ((269175.944 7032467.305, 269169.226 7...",825,5,211.333333,97.0,21
546,557,PV7705,"POLYGON ((273590.803 7041158.359, 273606.362 7...",507,11,189.000000,161.0,6
547,558,PV7872,"POLYGON ((266385.795 7032420.268, 266369.019 7...",406,9,171.538462,109.0,13


In [141]:
#Plotter de første 5
tmp = brattsving3.sort_values( ['Radius_min'], ascending=True)
tmp.iloc[0:6].hvplot(  crs=ccrs.UTM(zone=33), 
                     tiles='OSM', hover_cols=['vegnr', 'Radius_min', 'Radius_median'])

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]   (vegnr,Radius_min,Radius_median)

In [142]:
#Oppretter en tabell for hver vegkategori og sorterer denne slik at den minste radiusen kommer først
tabell = brattsving3.copy()
tabell['vegkategori'] = brattsving3['vegnr'].apply(lambda x: x[0])
#europaveg = tabell[tabell['vegkategori'] == 'E' or tabell['vegkategori'] == 'R']
europaveg = tabell[tabell['vegkategori'] == 'E']
europaveg_sort = europaveg.sort_values(['Radius_min'], ascending=True )
riksveg = tabell[tabell['vegkategori'] == 'R']
riksveg_sort = riksveg.sort_values(['Radius_min'], ascending=True )
fylkesveg = tabell[tabell['vegkategori'] == 'F']
fylkesveg_sort = fylkesveg.sort_values(['Radius_min'], ascending=True )
kommunalveg = tabell[tabell['vegkategori'] == 'K']
kommunalveg_sort = kommunalveg.sort_values(['Radius_min'], ascending=True )

# Statistikk over krappe svinger - europaveger

In [143]:
europaveg_sort

,minFineId,vegnr,geometry,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall,vegkategori
2,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7...",1115,10,227.518519,95.0,27,E
3,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7...",1624,11,382.000000,198.0,20,E
4,4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7...",1545,11,434.100000,122.5,10,E
5,5,EV6,"POLYGON ((275004.133 7041318.935, 274984.639 7...",3097,13,390.295775,190.0,71,E
1,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7...",1173,25,202.166667,87.0,36,E
0,0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7...",3704,39,395.714286,197.0,21,E


# Statistikk over krappe svinger - riksveg

In [144]:
riksveg_sort

,minFineId,vegnr,geometry,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall,vegkategori


# Statistikk over krappe svinger - fylkesveg

In [145]:
fylkesveg_sort

,minFineId,vegnr,geometry,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall,vegkategori
59,63,FV6650,"POLYGON ((268929.902 7040632.257, 268928.902 7...",9999,4,567.772152,88.0,79,F
145,149,FV6664,"POLYGON ((271842.007 7040304.673, 271850.298 7...",1968,9,272.487179,119.0,39,F
204,211,FV6696,"POLYGON ((272189.150 7030841.015, 272197.964 7...",2216,10,249.000000,127.5,32,F
133,137,FV6674,"POLYGON ((271885.100 7033300.439, 271883.541 7...",3509,10,422.750000,132.5,24,F
139,143,FV6662,"POLYGON ((273157.612 7038886.034, 273156.376 7...",599,10,165.857143,106.0,28,F
...,...,...,...,...,...,...,...,...,...
81,85,FV707,"POLYGON ((262912.967 7032820.385, 262929.198 7...",413,162,287.500000,287.5,2,F
13,15,FV6660,"POLYGON ((278954.403 7035924.390, 278936.332 7...",432,165,298.500000,298.5,2,F
14,16,FV6660,"POLYGON ((279326.225 7036392.795, 279342.690 7...",450,450,450.000000,450.0,1,F
91,95,FV6654,"POLYGON ((259029.612 7035664.959, 259027.676 7...",535,535,535.000000,535.0,1,F


# Statistikk over krappe svinger - kommunalveg

In [146]:
kommunalveg_sort

,minFineId,vegnr,geometry,Radius_max,Radius_min,Radius_mean,Radius_median,Radius_antall,vegkategori
191,198,KV6418,"POLYGON ((268108.024 7034572.617, 268125.870 7...",2739,1,238.190476,51.0,21,K
352,360,KV8713,"POLYGON ((271912.554 7037588.734, 271897.054 7...",1591,1,142.052632,43.5,38,K
428,437,KV8741,"POLYGON ((272529.262 7038226.582, 272530.812 7...",742,1,105.350000,41.0,40,K
310,318,KV5435,"POLYGON ((267828.632 7038064.120, 267828.593 7...",1049,2,232.028571,71.0,35,K
62,66,KV1184,"POLYGON ((269429.665 7039981.870, 269429.659 7...",6521,2,315.426087,92.0,115,K
...,...,...,...,...,...,...,...,...,...
472,482,KV2738,"POLYGON ((270130.042 7033564.944, 270117.279 7...",271,159,229.000000,243.0,4,K
136,140,KV5730,"POLYGON ((277427.990 7038972.317, 277447.503 7...",222,222,222.000000,222.0,1,K
451,461,KV1988,"POLYGON ((264826.237 7041973.411, 264820.909 7...",308,308,308.000000,308.0,1,K
160,164,KV6630,"POLYGON ((271734.919 7040529.282, 271734.939 7...",2418,2418,2418.000000,2418.0,1,K


# Steg 7 - gjør det samme som i steg 6, men nå med stigning

In [147]:
stig['Stigning_abs'] = stig['Stigning'].abs()
#Legger til kolonner (statistikk) for stigning
stig['Stigning_max']    = stig['Stigning_abs']
stig['Stigning_min']    = stig['Stigning_abs']
stig['Stigning_mean']   = stig['Stigning_abs']
stig['Stigning_median'] = stig['Stigning_abs']
stig['Stigning_antall'] = stig['Stigning_abs']
stig

,objekttype,nvdbId,versjon,startdato,Stigning,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,trafikantgruppe,geometri,Beregningsinfo,geometry,Stigning_abs,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall
0,825,323163525,2,2013-12-01,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1842939,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(269611.526 7040326.083 16.246, 26...",NaN,"LINESTRING Z (269611.526 7040326.083 16.246, 2...",6.5,6.5,6.5,6.5,6.5,6.5
1,825,323163525,2,2013-12-01,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1842954,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(269610.203 7040281.275 18.93, 269...",NaN,"LINESTRING Z (269610.203 7040281.275 18.930, 2...",6.5,6.5,6.5,6.5,6.5,6.5
2,825,323164319,2,2021-06-25,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",41468,Vegtrase,Kanalisert veg,5001,...,K,"LINESTRING Z(269548.1 7039562.1 25.637, 269547...",NaN,"LINESTRING Z (269548.100 7039562.100 25.637, 2...",6.5,6.5,6.5,6.5,6.5,6.5
3,825,323164448,1,2012-02-13,8.6,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",42837,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(281977.826 7032344.369 168.667, 2...",NaN,"LINESTRING Z (281977.826 7032344.369 168.667, ...",8.6,8.6,8.6,8.6,8.6,8.6
4,825,323164451,1,2012-02-13,10.7,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",42837,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(282011.62 7032168.76 177.634, 282...",NaN,"LINESTRING Z (282011.620 7032168.760 177.634, ...",10.7,10.7,10.7,10.7,10.7,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,825,1013893520,1,2021-08-04,7.8,NaN,45781,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(271202.675 7039959.71 51.819, 271...","bd=20210804,kv=3.0.0.20, as=0,8","LINESTRING Z (271202.675 7039959.710 51.819, 2...",7.8,7.8,7.8,7.8,7.8,7.8
1024,825,1013893558,1,2021-08-04,6.6,NaN,42681,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(267008.877 7038666.82 208.028, 26...","bd=20210804,kv=3.0.0.20, as=0,8","LINESTRING Z (267008.877 7038666.820 208.028, ...",6.6,6.6,6.6,6.6,6.6,6.6
1025,825,1013893561,1,2021-08-04,7.9,NaN,42681,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(267203.186 7038459.808 192.554, 2...","bd=20210804,kv=3.0.0.20, as=0,8","LINESTRING Z (267203.186 7038459.808 192.554, ...",7.9,7.9,7.9,7.9,7.9,7.9
1026,825,1013893562,1,2021-08-04,6.9,NaN,42681,Vegtrase og kjørebane,Enkel bilveg,5001,...,K,"LINESTRING Z(267309.396 7038457.627 184.998, 2...","bd=20210804,kv=3.0.0.20, as=0,8","LINESTRING Z (267309.396 7038457.627 184.998, ...",6.9,6.9,6.9,6.9,6.9,6.9


In [148]:
stigning_m_flatedata = gpd.sjoin( stig, brattsving2, how='inner', op='within'  )
stigning_m_flatedata

,objekttype,nvdbId,versjon,startdato,Stigning,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,geometry,Stigning_abs,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall,index_right,minFineId,vegnr
0,825,323163525,2,2013-12-01,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1842939,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (269611.526 7040326.083 16.246, 2...",6.5,6.5,6.5,6.5,6.5,6.5,77,77,KV5720
1,825,323163525,2,2013-12-01,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",1842954,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (269610.203 7040281.275 18.930, 2...",6.5,6.5,6.5,6.5,6.5,6.5,77,77,KV5720
2,825,323164319,2,2021-06-25,6.5,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",41468,Vegtrase,Kanalisert veg,5001,...,"LINESTRING Z (269548.100 7039562.100 25.637, 2...",6.5,6.5,6.5,6.5,6.5,6.5,66,66,KV1184
97,825,323209405,1,2012-01-25,9.4,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",41411,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (269475.649 7039695.669 31.954, 2...",9.4,9.4,9.4,9.4,9.4,9.4,66,66,KV1184
98,825,323209405,1,2012-01-25,9.4,"{'barn': [{'id': 201895, 'type': {'id': 641, '...",41411,Vegtrase,Kanalisert veg,5001,...,"LINESTRING Z (269253.100 7039849.600 61.236, 2...",9.4,9.4,9.4,9.4,9.4,9.4,66,66,KV1184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,825,1013892686,1,2021-08-04,8.2,NaN,41285,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (276161.874 7041390.163 17.856, 2...",8.2,8.2,8.2,8.2,8.2,8.2,293,293,KV1035
1019,825,1013892694,1,2021-08-04,7.7,NaN,2235532,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (276070.286 7041414.652 12.069, 2...",7.7,7.7,7.7,7.7,7.7,7.7,293,293,KV1035
1020,825,1013893092,1,2021-08-04,6.9,NaN,1205226,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (275808.760 7041752.910 6.898, 27...",6.9,6.9,6.9,6.9,6.9,6.9,37,37,KV3905
1021,825,1013893092,1,2021-08-04,6.9,NaN,1205239,Vegtrase og kjørebane,Enkel bilveg,5001,...,"LINESTRING Z (275780.184 7041718.925 9.779, 27...",6.9,6.9,6.9,6.9,6.9,6.9,37,37,KV3905


In [149]:
mineAggregeringsFunksjoner1 = {  'Stigning_max' : 'max', 
                                'Stigning_min' : 'min', 
                                'Stigning_mean' : 'mean', 
                                'Stigning_median' : 'median', 
                                'Stigning_antall' : 'count'
}

#Grupperer på minFineID
gruppert_per_flate_stigning = stigning_m_flatedata.groupby( ['minFineId'] ).agg( mineAggregeringsFunksjoner1 )

#Grupperer på vegnr og minFineID (tabellen som vises under)
stigning_m_flatedata.groupby( ['vegnr', 'minFineId']).agg( mineAggregeringsFunksjoner1 )

Stigning_max  Stigning_min  Stigning_mean  Stigning_median  \
vegnr   minFineId                                                               
EV6     1                   6.2           6.2       6.200000              6.2   
        2                  11.1           6.8       9.500000             10.6   
        3                   6.4           6.4       6.400000              6.4   
        5                   6.8           6.1       6.475000              6.5   
FV6610  62                 12.7           9.0      11.433333             12.6   
...                         ...           ...            ...              ...   
PV8117  204                 7.1           7.1       7.100000              7.1   
PV8120  337                15.0           6.6       9.587500              9.5   
PV8707  477                 7.5           7.1       7.300000              7.3   
PV8721  338                 9.9           6.7       8.300000              8.3   
PV95983 559                12.6           6.8       9.700000              9.7   

                   Stigning_antall  
vegnr   minFineId                   
EV6     1                        1  
        2                        3  
        3                        1  
        5                        8  
FV6610  62                       3  
...                            ...  
PV8117  204                      2  
PV8120  337                      8  
PV8707  477                      2  
PV8721  338                      2  
PV95983 559                      2  

[521 rows x 5 columns]

In [150]:
brattsving4 = brattsving2.merge( gruppert_per_flate_stigning, on='minFineId' )
tmp = brattsving4.sort_values( ['Stigning_max'], ascending=False)
tmp

,minFineId,vegnr,geometry,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall
21,30,KV1819,"POLYGON ((268306.216 7036532.024, 268294.181 7...",25.1,6.1,11.160000,7.50,5
280,312,KV1045,"POLYGON ((267872.822 7036147.368, 267860.522 7...",25.1,6.1,10.866667,10.70,9
22,31,KV2040,"POLYGON ((268316.549 7036660.842, 268315.499 7...",25.1,6.1,9.875000,8.85,12
43,53,KV6540,"POLYGON ((268440.899 7038082.337, 268460.434 7...",22.1,12.6,17.350000,17.35,2
467,504,KV5885,"POLYGON ((266756.151 7033492.577, 266747.444 7...",21.3,6.4,11.966667,8.20,3
...,...,...,...,...,...,...,...,...
457,494,KV8113,"POLYGON ((266861.728 7035350.266, 266862.740 7...",6.1,6.1,6.100000,6.10,1
119,138,KV5730,"POLYGON ((276430.897 7037902.070, 276439.806 7...",6.1,6.1,6.100000,6.10,1
129,151,FV6664,"POLYGON ((273378.182 7042237.840, 273397.073 7...",6.1,6.1,6.100000,6.10,2
434,471,KV4129,"POLYGON ((267444.469 7037662.451, 267463.173 7...",6.1,6.1,6.100000,6.10,1


In [151]:
#tmp.iloc[0:6].hvplot(  crs=ccrs.UTM(zone=33), 
 #                    tiles='OSM', hover_cols=['vegnr', 'Stigning_max', 'Stigning_median'])

tmp.iloc[0:5].hvplot(crs=ccrs.UTM(zone=33), colorbar=False, tiles='OSM', line_width=2 )

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [152]:
tmp.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='stigning_TEST', driver='GPKG')

# Statistikk over stigning - europaveger

In [153]:
sortertveg = brattsving4.copy()
sortertveg['vegkategori'] = brattsving4['vegnr'].apply(lambda x: x[0])
europaveg = sortertveg[ sortertveg['vegkategori'] == 'E' ]
e = europaveg.sort_values( ['Stigning_max'], ascending=False)
e

,minFineId,vegnr,geometry,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall,vegkategori
1,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7...",11.1,6.8,9.500,10.6,3,E
3,5,EV6,"POLYGON ((275004.133 7041318.935, 274984.639 7...",6.8,6.1,6.475,6.5,8,E
2,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7...",6.4,6.4,6.400,6.4,1,E
0,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7...",6.2,6.2,6.200,6.2,1,E


# Statistikk over stigning - riksveger

In [154]:
riksveg = sortertveg[ sortertveg['vegkategori'] == 'R' ]
r = riksveg.sort_values( ['Stigning_max'], ascending=False)
r

,minFineId,vegnr,geometry,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall,vegkategori


# Statistikk over stigning - fylkesveger

In [155]:
fylkesveg = sortertveg[ sortertveg['vegkategori'] == 'F' ]
f = fylkesveg.sort_values( ['Stigning_max'], ascending=False)
f

,minFineId,vegnr,geometry,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall,vegkategori
4,7,FV6658,"POLYGON ((271668.180 7039204.327, 271670.419 7...",16.4,6.3,10.075000,8.80,4,F
181,210,FV6696,"POLYGON ((272079.736 7029281.127, 272070.398 7...",15.1,7.3,10.833333,10.10,3,F
14,23,FV6682,"POLYGON ((267641.282 7032950.322, 267660.609 7...",14.9,14.9,14.900000,14.90,1,F
114,131,FV6674,"POLYGON ((279230.286 7031204.935, 279215.011 7...",13.7,6.2,9.000000,8.55,10,F
180,209,FV6696,"POLYGON ((272506.858 7029195.355, 272490.748 7...",13.7,11.5,12.600000,12.60,2,F
52,63,FV6650,"POLYGON ((268929.902 7040632.257, 268928.902 7...",13.5,6.1,8.750000,8.05,12,F
51,62,FV6610,"POLYGON ((268251.194 7029743.864, 268261.015 7...",12.7,9.0,11.433333,12.60,3,F
84,100,FV6656,"POLYGON ((269315.162 7038876.630, 269333.656 7...",12.7,6.1,8.237500,7.20,8,F
85,101,FV6656,"POLYGON ((269352.589 7039611.603, 269358.527 7...",12.7,6.5,8.866667,7.40,3,F
178,207,FV6696,"POLYGON ((272483.650 7028669.794, 272490.383 7...",12.7,7.1,10.700000,12.30,3,F


# Statistikk over stigning - kommunalveg

In [156]:
kommunalveg = sortertveg[ sortertveg['vegkategori'] == 'K' ]
k = kommunalveg.sort_values( ['Stigning_max'], ascending=False)
k

,minFineId,vegnr,geometry,Stigning_max,Stigning_min,Stigning_mean,Stigning_median,Stigning_antall,vegkategori
21,30,KV1819,"POLYGON ((268306.216 7036532.024, 268294.181 7...",25.1,6.1,11.160000,7.50,5,K
22,31,KV2040,"POLYGON ((268316.549 7036660.842, 268315.499 7...",25.1,6.1,9.875000,8.85,12,K
280,312,KV1045,"POLYGON ((267872.822 7036147.368, 267860.522 7...",25.1,6.1,10.866667,10.70,9,K
43,53,KV6540,"POLYGON ((268440.899 7038082.337, 268460.434 7...",22.1,12.6,17.350000,17.35,2,K
464,501,KV3465,"POLYGON ((267126.425 7033567.354, 267141.015 7...",21.3,6.4,12.057143,10.50,7,K
...,...,...,...,...,...,...,...,...,...
163,190,KV4690,"POLYGON ((267498.045 7032919.594, 267491.662 7...",6.1,6.1,6.100000,6.10,1,K
457,494,KV8113,"POLYGON ((266861.728 7035350.266, 266862.740 7...",6.1,6.1,6.100000,6.10,1,K
434,471,KV4129,"POLYGON ((267444.469 7037662.451, 267463.173 7...",6.1,6.1,6.100000,6.10,1,K
458,495,KV9412,"POLYGON ((267246.347 7034522.706, 267239.039 7...",6.1,6.1,6.100000,6.10,1,K


# Trafikkmengde

In [157]:
#Gjør videre det samme som over, men for stigningsdata. Dvs. henter objekttype for stigning, filtrerer på kommune, oppretter en 
#tabell.

sokeobjekt_trafikkmengde = nvdbapiv3.nvdbFagdata( 540 )
#sokeobjekt_trafikkmengde.filter(  { 'egenskap' : '9396 > 6 OR 9396 < -6'  }  )
sokeobjekt_trafikkmengde.filter( {'kommune' : 1560 })
trafikkmengde = pd.DataFrame( sokeobjekt_trafikkmengde.to_records( ) )

#Oppretter geodataframe og lagrer til fil
trafikkmengde['geometry'] = trafikkmengde['geometri'].apply( lambda x : wkt.loads( x ))
trafikkmengde = gpd.GeoDataFrame( trafikkmengde, geometry='geometry', crs=5973 )
trafikkmengde.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='trafikkmengde_ALT', driver='GPKG' )
trafikkmengde3 = trafikkmengde[ trafikkmengde['ÅDT, total'] >= 1000]
trafikkmengde3.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='trafikkmengde_>=1000', driver='GPKG' )

#Lager buffer rundt områder med stor stigning (+- 6 %) (Buffer har radius 500 - samme som buffer rundt krappe svinger (steg 2))
trafikkbuffer = trafikkmengde.copy()
trafikkbuffer['geometry'] = trafikkbuffer['geometry'].buffer( 200 ) # 200 meters radius

trafikkbuffer3 = trafikkmengde3.copy()
trafikkbuffer3['geometry'] = trafikkbuffer3['geometry'].buffer( 200 ) # 200 meters radius

#Splitter opp vegnummer på mellomrom
trafikkbuffer['vegnr'] = trafikkbuffer['vref'].apply( lambda x: x.split()[0] )

trafikkbuffer_dissolve = trafikkbuffer.dissolve( by='vegnr')
trafikkbuffer_dissolve.reset_index( inplace=True )

trafikkbuffer_dissolve2 = trafikkbuffer_dissolve.explode( )

trafikkbuffer_dissolve2.reset_index( inplace=True, drop=True )

#Lager buffer rundt alle veier med trafikkmengdedata
#trafikkbuffer_dissolve2.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='trafikkbuffer2', driver='GPKG' )

C:\Users\Kristin\OneDrive\nvdbapi-V3-master\nvdbapiv3\nvdbapiv3.py:438: UserWarning: 
You should provide the file nvdbapi-clientinfo.json 
 
{ "X-Client" : "YOUR SYSTEM",
 "X-Kontaktperson" : "ola.nordmann@eposten.din" }

  warn( mytext )


In [158]:
#Splitter opp vegnummer på mellomrom
trafikkbuffer3['vegnr'] = trafikkbuffer3['vref'].apply( lambda x: x.split()[0] )

trafikkbuffer_dissolve3 = trafikkbuffer3.dissolve( by='vegnr')
trafikkbuffer_dissolve3.reset_index( inplace=True )

trafikkbuffer_dissolve4 = trafikkbuffer_dissolve3.explode( )

trafikkbuffer_dissolve4.reset_index( inplace=True, drop=True )

#Lager buffer rundt alle veier med trafikkmengdedata
trafikkbuffer_dissolve4.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='trafikkbuffer3', driver='GPKG' )

In [159]:
trafikkmengde

,objekttype,nvdbId,versjon,startdato,Historisk_Ansvarlig ID,"År, gjelder for","ÅDT, total","ÅDT, andel lange kjøretøy",Grunnlag for ÅDT,veglenkesekvensid,...,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Historisk_Mfaktor,"ÅDT, start","ÅDT, slutt",geometry
0,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,249592,...,0.161573,0.331839,1691.965,Nei,K,"LINESTRING Z(155941.07 6987646.87 123.9, 15593...",NaN,NaN,NaN,"LINESTRING Z (155941.070 6987646.870 123.900, ..."
1,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,249592,...,0.348921,0.372749,234.000,Nei,K,"LINESTRING Z(155008.882 6988517.809 122.391, 1...",NaN,NaN,NaN,"LINESTRING Z (155008.882 6988517.809 122.391, ..."
2,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,249592,...,0.385133,0.399293,141.000,Nei,K,"LINESTRING Z(155006.456 6988772.808 112.213, 1...",NaN,NaN,NaN,"LINESTRING Z (155006.456 6988772.808 112.213, ..."
3,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,249592,...,0.454485,0.459593,45.714,Nei,K,"LINESTRING Z(155360.52 6989342.64 93.25, 15537...",NaN,NaN,NaN,"LINESTRING Z (155360.520 6989342.640 93.250, 1..."
4,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,249592,...,0.506322,0.512583,61.488,Nei,K,"LINESTRING Z(155376.423 6989823.122 94.38, 155...",NaN,NaN,NaN,"LINESTRING Z (155376.423 6989823.122 94.380, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,3159996,...,0.438948,0.924861,26.270,Nei,K,"LINESTRING Z(158508.61 6986981.26 19.715, 1585...",NaN,NaN,NaN,"LINESTRING Z (158508.610 6986981.260 19.715, 1..."
86,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,249355,...,0.640724,1.000000,2457.481,Nei,K,"LINESTRING Z(160051.225 6989098.689 16.703, 16...",NaN,NaN,NaN,"LINESTRING Z (160051.225 6989098.689 16.703, 1..."
87,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,249591,...,0.854872,0.887495,323.461,Nei,K,"LINESTRING Z(158504.9 6987004.4 20.045, 158505...",NaN,NaN,NaN,"LINESTRING Z (158504.900 6987004.400 20.045, 1..."
88,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,3159996,...,0.924861,1.000000,4.000,Nei,K,"LINESTRING Z(158515.534 6986956.147 20.049, 15...",NaN,NaN,NaN,"LINESTRING Z (158515.534 6986956.147 20.049, 1..."


In [160]:
trafikkmengde.reset_index( inplace=True )
trafikkmengde.rename( columns={'index' : 'minFineId'}, inplace=True)
trafikkmengde

,minFineId,objekttype,nvdbId,versjon,startdato,Historisk_Ansvarlig ID,"År, gjelder for","ÅDT, total","ÅDT, andel lange kjøretøy",Grunnlag for ÅDT,...,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Historisk_Mfaktor,"ÅDT, start","ÅDT, slutt",geometry
0,0,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,...,0.161573,0.331839,1691.965,Nei,K,"LINESTRING Z(155941.07 6987646.87 123.9, 15593...",NaN,NaN,NaN,"LINESTRING Z (155941.070 6987646.870 123.900, ..."
1,1,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,...,0.348921,0.372749,234.000,Nei,K,"LINESTRING Z(155008.882 6988517.809 122.391, 1...",NaN,NaN,NaN,"LINESTRING Z (155008.882 6988517.809 122.391, ..."
2,2,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,...,0.385133,0.399293,141.000,Nei,K,"LINESTRING Z(155006.456 6988772.808 112.213, 1...",NaN,NaN,NaN,"LINESTRING Z (155006.456 6988772.808 112.213, ..."
3,3,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,...,0.454485,0.459593,45.714,Nei,K,"LINESTRING Z(155360.52 6989342.64 93.25, 15537...",NaN,NaN,NaN,"LINESTRING Z (155360.520 6989342.640 93.250, 1..."
4,4,540,80275938,17,2020-12-31,TG,2018,1060,11,NorTraf,...,0.506322,0.512583,61.488,Nei,K,"LINESTRING Z(155376.423 6989823.122 94.38, 155...",NaN,NaN,NaN,"LINESTRING Z (155376.423 6989823.122 94.380, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,...,0.438948,0.924861,26.270,Nei,K,"LINESTRING Z(158508.61 6986981.26 19.715, 1585...",NaN,NaN,NaN,"LINESTRING Z (158508.610 6986981.260 19.715, 1..."
86,86,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,...,0.640724,1.000000,2457.481,Nei,K,"LINESTRING Z(160051.225 6989098.689 16.703, 16...",NaN,NaN,NaN,"LINESTRING Z (160051.225 6989098.689 16.703, 1..."
87,87,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,...,0.854872,0.887495,323.461,Nei,K,"LINESTRING Z(158504.9 6987004.4 20.045, 158505...",NaN,NaN,NaN,"LINESTRING Z (158504.900 6987004.400 20.045, 1..."
88,88,540,1013759419,1,2021-03-05,NaN,2020,300,10,NaN,...,0.924861,1.000000,4.000,Nei,K,"LINESTRING Z(158515.534 6986956.147 20.049, 15...",NaN,NaN,NaN,"LINESTRING Z (158515.534 6986956.147 20.049, 1..."


In [161]:
#Bruker minFineID som koblingsnøkler når man merger tabellene
trafikkmengde2 = brattsving2.merge( trafikkmengde, on='minFineId' )
trafikkmengde2

,minFineId,vegnr,geometry_x,objekttype,nvdbId,versjon,startdato,Historisk_Ansvarlig ID,"År, gjelder for","ÅDT, total",...,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Historisk_Mfaktor,"ÅDT, start","ÅDT, slutt",geometry_y
0,0,EV6,"POLYGON ((269127.717 7036516.223, 269116.375 7...",540,80275938,17,2020-12-31,TG,2018,1060,...,0.161573,0.331839,1691.965,Nei,K,"LINESTRING Z(155941.07 6987646.87 123.9, 15593...",NaN,NaN,NaN,"LINESTRING Z (155941.070 6987646.870 123.900, ..."
1,1,EV6,"POLYGON ((272206.098 7039305.599, 272206.105 7...",540,80275938,17,2020-12-31,TG,2018,1060,...,0.348921,0.372749,234.000,Nei,K,"LINESTRING Z(155008.882 6988517.809 122.391, 1...",NaN,NaN,NaN,"LINESTRING Z (155008.882 6988517.809 122.391, ..."
2,2,EV6,"POLYGON ((271901.165 7039359.246, 271917.294 7...",540,80275938,17,2020-12-31,TG,2018,1060,...,0.385133,0.399293,141.000,Nei,K,"LINESTRING Z(155006.456 6988772.808 112.213, 1...",NaN,NaN,NaN,"LINESTRING Z (155006.456 6988772.808 112.213, ..."
3,3,EV6,"POLYGON ((277327.849 7040564.693, 277339.828 7...",540,80275938,17,2020-12-31,TG,2018,1060,...,0.454485,0.459593,45.714,Nei,K,"LINESTRING Z(155360.52 6989342.64 93.25, 15537...",NaN,NaN,NaN,"LINESTRING Z (155360.520 6989342.640 93.250, 1..."
4,4,EV6,"POLYGON ((276787.053 7040406.475, 276783.541 7...",540,80275938,17,2020-12-31,TG,2018,1060,...,0.506322,0.512583,61.488,Nei,K,"LINESTRING Z(155376.423 6989823.122 94.38, 155...",NaN,NaN,NaN,"LINESTRING Z (155376.423 6989823.122 94.380, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,FV707,"POLYGON ((262912.967 7032820.385, 262929.198 7...",540,1013759419,1,2021-03-05,NaN,2020,300,...,0.438948,0.924861,26.270,Nei,K,"LINESTRING Z(158508.61 6986981.26 19.715, 1585...",NaN,NaN,NaN,"LINESTRING Z (158508.610 6986981.260 19.715, 1..."
86,86,FV707,"POLYGON ((258492.725 7033630.264, 258495.227 7...",540,1013759419,1,2021-03-05,NaN,2020,300,...,0.640724,1.000000,2457.481,Nei,K,"LINESTRING Z(160051.225 6989098.689 16.703, 16...",NaN,NaN,NaN,"LINESTRING Z (160051.225 6989098.689 16.703, 1..."
87,87,FV707,"POLYGON ((257713.247 7041715.293, 257721.730 7...",540,1013759419,1,2021-03-05,NaN,2020,300,...,0.854872,0.887495,323.461,Nei,K,"LINESTRING Z(158504.9 6987004.4 20.045, 158505...",NaN,NaN,NaN,"LINESTRING Z (158504.900 6987004.400 20.045, 1..."
88,88,KV3726,"POLYGON ((258318.157 7042365.594, 258321.415 7...",540,1013759419,1,2021-03-05,NaN,2020,300,...,0.924861,1.000000,4.000,Nei,K,"LINESTRING Z(158515.534 6986956.147 20.049, 15...",NaN,NaN,NaN,"LINESTRING Z (158515.534 6986956.147 20.049, 1..."


In [162]:
ådt_tot = trafikkmengde2[ trafikkmengde2['ÅDT, total'] >= 1000]
ådt_sort = ådt_tot.sort_values( ['ÅDT, total'], ascending=False)
ådt_sort

,minFineId,vegnr,geometry_x,objekttype,nvdbId,versjon,startdato,Historisk_Ansvarlig ID,"År, gjelder for","ÅDT, total",...,startposisjon,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Historisk_Mfaktor,"ÅDT, start","ÅDT, slutt",geometry_y
49,49,KV6283,"POLYGON ((268012.394 7035826.991, 268000.486 7...",540,1013753147,1,2021-03-05,NaN,2020,3595,...,0.566368,0.856363,2827.433,Nei,K,"LINESTRING Z(139462.793 7004163.17 11.423, 139...",NaN,NaN,NaN,"LINESTRING Z (139462.793 7004163.170 11.423, 1..."
54,54,KV6560,"POLYGON ((276548.956 7040070.743, 276544.769 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,0.856363,1.000000,1399.159,Nei,K,"LINESTRING Z(141994.098 7004798.145 4.031, 142...",NaN,NaN,NaN,"LINESTRING Z (141994.098 7004798.145 4.031, 14..."
53,53,KV6540,"POLYGON ((268440.899 7038082.337, 268460.434 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,0.315819,0.330447,145.406,Nei,K,"LINESTRING Z(146017 7006250 31.32, 146028.9 70...",NaN,NaN,NaN,"LINESTRING Z (146017.000 7006250.000 31.320, 1..."
52,52,KV6540,"POLYGON ((269076.333 7037054.125, 269059.131 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,0.000000,0.315819,3139.180,Nei,K,"LINESTRING Z(143297.91 7005269.852 49.71, 1433...",NaN,NaN,NaN,"LINESTRING Z (143297.910 7005269.852 49.710, 1..."
33,33,KV2905,"POLYGON ((271862.940 7038848.206, 271865.340 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.339358,0.745547,4028.910,Nei,K,"LINESTRING Z(146182.799 7006268.988 36.11, 146...",NaN,NaN,NaN,"LINESTRING Z (146182.799 7006268.988 36.110, 1..."
35,35,KV3530,"POLYGON ((271009.243 7040673.672, 270993.674 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.334120,0.339358,56.892,Nei,K,"LINESTRING Z(146162.389 7006217.094 38.52, 146...",NaN,NaN,NaN,"LINESTRING Z (146162.389 7006217.094 38.520, 1..."
34,34,KV3330,"POLYGON ((275082.064 7041029.247, 275079.112 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.330930,0.334120,31.750,Nei,K,"LINESTRING Z(146149.125 7006188.45 38.158, 146...",NaN,NaN,NaN,"LINESTRING Z (146149.125 7006188.450 38.158, 1..."
47,47,KV6130,"POLYGON ((278576.886 7041157.364, 278572.225 7...",540,1013752526,1,2021-03-05,NaN,2020,1600,...,0.000000,1.000000,2097.595,Nei,K,"LINESTRING Z(150902.147 7009924.234 25.681, 15...",NaN,NaN,NaN,"LINESTRING Z (150902.147 7009924.234 25.681, 1..."
48,48,KV6230,"POLYGON ((268580.687 7036720.655, 268575.633 7...",540,1013752526,1,2021-03-05,NaN,2020,1600,...,0.745547,1.000000,2532.662,Nei,K,"LINESTRING Z(148752.934 7009033.868 25.19, 148...",NaN,NaN,NaN,"LINESTRING Z (148752.934 7009033.868 25.190, 1..."
5,5,EV6,"POLYGON ((275004.133 7041318.935, 274984.639 7...",540,80282346,2,1988-02-15,wjp,1988,1500,...,0.903955,1.000000,17.000,NaN,NaN,"LINESTRING Z(164777.2 6980000.4 0, 164794.2 69...",0.0,1500.0,1500.0,"LINESTRING Z (164777.200 6980000.400 0.000, 16..."


In [163]:
ådt_sort.dtypes

minFineId                       int64
vegnr                          object
geometry_x                   geometry
objekttype                      int64
nvdbId                          int64
versjon                         int64
startdato                      object
Historisk_Ansvarlig ID         object
År, gjelder for                 int64
ÅDT, total                      int64
ÅDT, andel lange kjøretøy       int64
Grunnlag for ÅDT               object
veglenkesekvensid               int64
detaljnivå                     object
typeVeg                        object
kommune                         int64
fylke                           int64
vref                           object
veglenkeType                   object
vegkategori                    object
fase                           object
vegnummer                       int64
startposisjon                 float64
sluttposisjon                 float64
segmentlengde                 float64
adskilte_lop                   object
trafikantgru

In [164]:
ådt_sort[['vegnr', 'vref']]

,vegnr,vref
49,KV6283,EV39 S18D1 m4777-7605
54,KV6560,EV39 S18D1 m3378-4777
53,KV6540,EV39 S18D1 m93-239
52,KV6540,EV39 S18D1 m239-3378
33,KV2905,EV39 S17D1 m4630-8659
35,KV3530,EV39 S18D1 m0-57
34,KV3330,EV39 S18D1 m57-89
47,KV6130,EV39 S17D1 m0-2098
48,KV6230,EV39 S17D1 m2098-4630
5,EV6,EV69


In [165]:
ådt_sort['ådt_vegnr'] = ådt_sort['vref'].apply( lambda x: x.split()[0] )
ådt_sort

,minFineId,vegnr,geometry_x,objekttype,nvdbId,versjon,startdato,Historisk_Ansvarlig ID,"År, gjelder for","ÅDT, total",...,sluttposisjon,segmentlengde,adskilte_lop,trafikantgruppe,geometri,Historisk_Mfaktor,"ÅDT, start","ÅDT, slutt",geometry_y,ådt_vegnr
49,49,KV6283,"POLYGON ((268012.394 7035826.991, 268000.486 7...",540,1013753147,1,2021-03-05,NaN,2020,3595,...,0.856363,2827.433,Nei,K,"LINESTRING Z(139462.793 7004163.17 11.423, 139...",NaN,NaN,NaN,"LINESTRING Z (139462.793 7004163.170 11.423, 1...",EV39
54,54,KV6560,"POLYGON ((276548.956 7040070.743, 276544.769 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,1.000000,1399.159,Nei,K,"LINESTRING Z(141994.098 7004798.145 4.031, 142...",NaN,NaN,NaN,"LINESTRING Z (141994.098 7004798.145 4.031, 14...",EV39
53,53,KV6540,"POLYGON ((268440.899 7038082.337, 268460.434 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,0.330447,145.406,Nei,K,"LINESTRING Z(146017 7006250 31.32, 146028.9 70...",NaN,NaN,NaN,"LINESTRING Z (146017.000 7006250.000 31.320, 1...",EV39
52,52,KV6540,"POLYGON ((269076.333 7037054.125, 269059.131 7...",540,1013753609,1,2021-03-05,NaN,2020,3000,...,0.315819,3139.180,Nei,K,"LINESTRING Z(143297.91 7005269.852 49.71, 1433...",NaN,NaN,NaN,"LINESTRING Z (143297.910 7005269.852 49.710, 1...",EV39
33,33,KV2905,"POLYGON ((271862.940 7038848.206, 271865.340 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.745547,4028.910,Nei,K,"LINESTRING Z(146182.799 7006268.988 36.11, 146...",NaN,NaN,NaN,"LINESTRING Z (146182.799 7006268.988 36.110, 1...",EV39
35,35,KV3530,"POLYGON ((271009.243 7040673.672, 270993.674 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.339358,56.892,Nei,K,"LINESTRING Z(146162.389 7006217.094 38.52, 146...",NaN,NaN,NaN,"LINESTRING Z (146162.389 7006217.094 38.520, 1...",EV39
34,34,KV3330,"POLYGON ((275082.064 7041029.247, 275079.112 7...",540,1013747790,1,2021-03-05,NaN,2020,2400,...,0.334120,31.750,Nei,K,"LINESTRING Z(146149.125 7006188.45 38.158, 146...",NaN,NaN,NaN,"LINESTRING Z (146149.125 7006188.450 38.158, 1...",EV39
47,47,KV6130,"POLYGON ((278576.886 7041157.364, 278572.225 7...",540,1013752526,1,2021-03-05,NaN,2020,1600,...,1.000000,2097.595,Nei,K,"LINESTRING Z(150902.147 7009924.234 25.681, 15...",NaN,NaN,NaN,"LINESTRING Z (150902.147 7009924.234 25.681, 1...",EV39
48,48,KV6230,"POLYGON ((268580.687 7036720.655, 268575.633 7...",540,1013752526,1,2021-03-05,NaN,2020,1600,...,1.000000,2532.662,Nei,K,"LINESTRING Z(148752.934 7009033.868 25.19, 148...",NaN,NaN,NaN,"LINESTRING Z (148752.934 7009033.868 25.190, 1...",EV39
5,5,EV6,"POLYGON ((275004.133 7041318.935, 274984.639 7...",540,80282346,2,1988-02-15,wjp,1988,1500,...,1.000000,17.000,NaN,NaN,"LINESTRING Z(164777.2 6980000.4 0, 164794.2 69...",0.0,1500.0,1500.0,"LINESTRING Z (164777.200 6980000.400 0.000, 16...",EV69


In [166]:
ådt_edit = ådt_tot[[ 'vegnr', 'ÅDT, total', 'geometri', 'geometry_x' ] ].copy()
ådt_edit

,vegnr,"ÅDT, total",geometri,geometry_x
0,EV6,1060,"LINESTRING Z(155941.07 6987646.87 123.9, 15593...","POLYGON ((269127.717 7036516.223, 269116.375 7..."
1,EV6,1060,"LINESTRING Z(155008.882 6988517.809 122.391, 1...","POLYGON ((272206.098 7039305.599, 272206.105 7..."
2,EV6,1060,"LINESTRING Z(155006.456 6988772.808 112.213, 1...","POLYGON ((271901.165 7039359.246, 271917.294 7..."
3,EV6,1060,"LINESTRING Z(155360.52 6989342.64 93.25, 15537...","POLYGON ((277327.849 7040564.693, 277339.828 7..."
4,EV6,1060,"LINESTRING Z(155376.423 6989823.122 94.38, 155...","POLYGON ((276787.053 7040406.475, 276783.541 7..."
5,EV6,1500,"LINESTRING Z(164777.2 6980000.4 0, 164794.2 69...","POLYGON ((275004.133 7041318.935, 274984.639 7..."
15,FV6660,1100,"LINESTRING Z(154814.34 6992205.47 84.456, 1548...","POLYGON ((278954.403 7035924.390, 278936.332 7..."
16,FV6660,1400,"LINESTRING Z(155367.1 6993743.6 14.436, 155361...","POLYGON ((279326.225 7036392.795, 279342.690 7..."
17,FV6660,1400,"LINESTRING Z(155344.17 6993804.344 13.636, 155...","POLYGON ((278593.488 7036291.051, 278608.549 7..."
22,FV6682,1300,"LINESTRING Z(147444.506 7004670.504 67.913, 14...","POLYGON ((265979.932 7030897.354, 265999.556 7..."


In [167]:
ådt_edit = gpd.GeoDataFrame( ådt_edit, geometry='geometry_x', crs=5973 )
#ådt_edit.to_file( 'kurvaturanalyse_arbeid_test.gpkg', layer='trafikkmengde_1000', driver='GPKG' )